<a href="https://colab.research.google.com/github/andersonsam/cnn_lstm_era/blob/master/main_publish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary

In [1]:
#download required libraries which are not in colab

!pip install geopandas
!pip install netCDF4
!pip install minisom
!pip install guppy3

     |████████████████████████████████| 972kB 8.1MB/s 
     |████████████████████████████████| 14.8MB 293kB/s 
     |████████████████████████████████| 10.9MB 39.2MB/s 
     |████████████████████████████████| 4.3MB 7.9MB/s 
     |████████████████████████████████| 296kB 31.8MB/s 
  Created wheel for minisom: filename=MiniSom-2.2.6-cp36-none-any.whl size=8525 sha256=2a1c2a450966e9d6e45096ce61ab04e7b8624ceb99861bea6947c2db4d9ccc45
  Stored in directory: /root/.cache/pip/wheels/b8/c6/01/330066e36e1f7c826c96f656f9185822cfcdef0591315949ea
Successfully built minisom
     |████████████████████████████████| 604kB 8.4MB/s 


In [ ]:
#load in functions required and dependencies to use them

%tensorflow_version 2.x
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense, Dropout, GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm#, colors, path
from mpl_toolkits.axes_grid.inset_locator import inset_axes, InsetPosition

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

from scipy import interpolate
from scipy.stats import ks_2samp
from sklearn.cluster import AgglomerativeClustering

import pickle
import os
from datetime import datetime, date
from netCDF4 import Dataset
from guppy import hpy
from google.colab import drive

from shapely.geometry import Point, Polygon
from descartes import PolygonPatch


In [ ]:
#mount google drive

drive.mount('/content/drive')

In [ ]:
#define functions

def nse(y_obs, y_model):

  """
  NSE = nse(y_obs, y_model)

  y_obs, y_model --> these are arrays of the same length (1 x N or N x 1) where N is the number of observations in time
  """

  if not isinstance(y_obs, np.ndarray): #if tensor, convert to numpy array
    y_obs = np.array(y_obs)
  if not isinstance(y_model, np.ndarray):
    y_model = np.array(y_model)

  y_model = y_model.reshape((-1,1)) #make sure model and obs have same shape
  y_obs = y_obs.reshape((-1,1))

  nse = 1 - np.sum((y_model - y_obs)**2) / np.sum((y_obs - np.mean(y_obs))**2) #calculate NSE

  return nse

def plot_prov_ax(prov, ax):

    """
    plot borders of a province on a given axis
    
    prov: string; 'AB', 'BC', or 'AB_BC'
    ax: axis on which to plot the provincial borders
    """
    
    if prov == 'AB_BC':
      provs = ['AB', 'BC']
      for prov in provs:
        if prov == 'AB':
          provIndex=0
        elif prov == 'BC':
          provIndex = 11
        provshapes_filename = '/content/drive/My Drive/Colab Notebooks/cnn_lstm_era/PROVINCE.SHP'
        provshapes = gpd.read_file(provshapes_filename)
        provPoly = provshapes['geometry'][provIndex]

        if len(np.shape(provPoly)) == 0: #if only one polygon to plot

          lonBorder,latBorder = provPoly.exterior.coords.xy 
          ax.plot(lonBorder,latBorder,'k')

        else: #if multiply polygons in shape to plot

          for ind in range(len(provPoly)):

            lonBorder_segment,latBorder_segment = provPoly[ind].exterior.coords.xy 
            ax.plot(lonBorder_segment,latBorder_segment,'k')

    else:
      if prov == 'AB':
        provIndex=0
      elif prov == 'BC':
        provIndex = 11
      provshapes_filename = '/content/drive/My Drive/Colab Notebooks/cnn_lstm_era/PROVINCE.SHP'
      provshapes = gpd.read_file(provshapes_filename)
      provPoly = provshapes['geometry'][provIndex]

      if len(np.shape(provPoly)) == 0: #if only one polygon to plot

        lonBorder,latBorder = provPoly.exterior.coords.xy 
        ax.plot(lonBorder,latBorder,'k')

      else: #if multiply polygons in shape to plot

        for ind in range(len(provPoly)):

          lonBorder_segment,latBorder_segment = provPoly[ind].exterior.coords.xy 
          ax.plot(lonBorder_segment,latBorder_segment,'k')

def rmse(target,prediction):

  """
  Returns the root-mean-square error between a target and prediction
  target, prediction: arrays or tensors of equal length

  Example: 
  RMSE = rmse(target,prediction) 
  """

  if not isinstance(target, np.ndarray):
    target = np.array(target)
  if not isinstance(prediction, np.ndarray):
    prediction = np.array(prediction)

  return(np.sqrt(((target.reshape(-1,1) - prediction.reshape(-1,1))**2).sum()/len(target.reshape(-1,1))))

def get_A(heat):

  """
  Returns the area (as a fraction of the total heatmap) which is above the half-maximum value

  Example:
  A = get_A(heat = heat_mean)
  """

  halfMax = 0.5* (np.max(heat) - np.min(heat))
  n_hot_pixels = len(np.argwhere((heat - np.min(heat)) > halfMax))
  n_pixels = np.shape(heat)[0]*np.shape(heat)[1]
  A = n_hot_pixels / n_pixels

  return A

def make_heat(model, x_test, y_test, style_dict, days, iters_total, iters_one_pass, output_dir, saveFiles, stationInds, verbose):

  """
  model: 
      .h5 keras model
  x_test:
      tf tensor; test set of ERA data, input to model (shape = Ntest x 365 x height x width x channels)
  y_test:
      tf tensor; test set of streamflow data, target output of model (shape = Ntest x Nstations)
  style_dict:
      dictionary: {'style' : 'RISE' or 'gauss',
                   'params' : [h,w,p_1] or sigma}
          where [h,w,p_1] are the height/width/probability of perturbation of low-res mask (for RISE algorithm); sigma is the gaussian RMS width
  days:
      range of days in test set to perturb (e.g. days = range(0,365) will perturb the first 365 days in the test set)
  iters_total:
      number of total iterations of perturbation to do for each day in days
  iters_one_pass:
      number of iterations to do at one time (typically less than iters_total for memory limits)
  output_dir:
      directory where output will be saved, string 
  save_files:
      0 or 1 (False or True) if output heat maps are to be saved to the output_dir or not
  stationInds:
      indices of stations corresponding to the output neurons of the keras model
  verbose:
      0: print nothing
      1: print every 50th day
      2: print every day and iteration
  """

  n_channels = np.shape(x_test)[-1] #number of channels of input video
  H = np.shape(x_test)[2] #height of input video, in pixels
  W = np.shape(x_test)[3] #width of input video, in pixels

  heat_all_slices = [[] for station in range(np.shape(y_test)[0])] #list of empty lists (one empty list per output station)
  heat_days = [[] for station in range(np.shape(y_test)[0])] #list of empty lists (one empty list per output station)
  jj = 0

  for day in days: #for each day in test set that we will perturb

    if verbose == 2 or (verbose == 1 and np.mod(day,50) == 0):
      print('Day ' + str(day) + '/' + str(days[-1])) 

    for kk in range(int(iters_total/iters_one_pass)): #for each batch of iterations 

      if verbose == 2:
        print('   Iteration: ' + str(kk*iters_one_pass) + '/' + str(iters_total))

      iters = iters_one_pass 

      if style_dict['style'] == 'RISE':
        
        h = style_dict['params'][0]
        w = style_dict['params'][1]
        p_1 = style_dict['params'][2]

        x_int = np.linspace(0,W,w) #low-res x indices
        y_int = np.linspace(0,H,h) #low-res y indices

        xnew = np.arange(W)
        ynew = np.arange(H) 

        perturb_small = np.random.choice([0,1],size = (iters,1,h,w), p = [1-p_1,p_1])
        perturb = np.half([interpolate.interp2d(x_int,y_int,perturb_small[iter][0])(xnew,ynew) for iter in range(iters)])

      elif style_dict['style'] == 'gauss':
        
        sigma = style_dict['params']

        x_int = np.arange(W)
        y_int = np.arange(H)

        x_mesh, y_mesh = np.meshgrid(x_int, y_int)
        pointx = np.random.randint(0,np.shape(T[0])[1])
        pointy = np.random.randint(0,np.shape(T[0])[0])
        d2 = (x_mesh - pointx)**2 + (y_mesh - pointy)**2
        perturb = np.half([np.exp( -d2 / (2*sigma**2)) for iter in range(iters)])

      perturb_2D = np.copy(perturb)
      perturb = tf.repeat(tf.expand_dims(tf.convert_to_tensor(perturb),3),nchannels, axis = 3)
      perturb = tf.repeat(tf.expand_dims(tf.convert_to_tensor(perturb),1),365, axis = 1)

      day_slice = [day]

      xday = x_test[day]
      xday_iters = [xday for val in range(iters)]

      factor = np.random.choice([-1,1],p = [0.5,0.5]) #whether to add or subtract perturbation from input video
      perturb = factor*perturb
      x1 = perturb
      x2 = tf.convert_to_tensor(xday_iters)
      xday_iters_mask = tf.math.add(x1,x2)#.numpy()

      x_all = tf.squeeze(tf.concat((tf.expand_dims(xday, axis = 0),xday_iters_mask), axis = 0))
      x_all_ds = tf.data.Dataset.from_tensor_slices(x_all).batch(batch_size = 64)
      y_all = model.predict(x_all_ds)

      yday = y_all[:len(day_slice)]
      yday_mask = y_all[len(day_slice):]

      for station in range(np.shape(y_all)[1]):

        yday_station = yday[:,station]
        yday_station_mask = yday_mask[:,station]

        ydiffs = np.abs(np.reshape([yday_station[jj] - yday_station_mask[jj*iters:jj*iters + iters] for jj in range(len(day_slice))],(-1,1)))
        delta = np.ones((len(ydiffs),H,W)) * ydiffs[:,None]
        heat_iters = [np.asarray(delta[jj*iters:(jj+1)*iters]) * np.asarray(perturb_2D) for jj in range(len(day_slice))]
        heat_iters = np.reshape(heat_iters,(len(day_slice)*iters,H,W))
        heat = [np.mean(heat_iters[jj*iters : (jj+1)*iters], axis=0) for jj in range(len(day_slice))] #fast
        
        heat_all_slices[station].append(heat[0]) #fast

      del heat, heat_iters, delta, ydiffs, x_all, xday_iters

    for station in range(np.shape(y_all)[1]):
      heat_days[station].append(np.mean(heat_all_slices[station][jj : jj + int(iters_total/iters_one_pass)], axis = 0))
    jj += int(iters_total/iters_one_pass)

    heat_mean = np.empty( (np.shape(y_all)[1] ,) + np.shape(np.mean(heat_all_slices[0],axis=0)) )
    for station in range(np.shape(y_all)[1]):
      heat_mean[station] = np.mean(heat_all_slices[station],axis=0)

  for zz, station in enumerate(stationInds):

    heat_days_vec = np.empty((len(days),np.size(x_test[0,0,:,:,0])))
    for day in days:
      heat_days_vec[day,:] = np.reshape(heat_days[zz][day],(1,-1))

    if saveFiles:

      fileName = 'heat_mean_station_' + str(station) + '.csv'
      fileName_days = 'heat_days_station_' + str(station) + '.csv'

      if not path.exists(output_dir):
        os.mkdir(output_dir)
      np.savetxt(output_dir + '/' + fileName, heat_mean[zz], delimiter = ',')
      #np.savetxt(output_dir + '/' + fileName_days, heat_days_vec, delimiter = ',')

  return heat_mean, heat_days_vec


# Prep data

In [ ]:
prov = 'AB_BC' #for plotting (plot_prov)
flowpickle = ['BC_flowvars_1979_2015.pickle', 'AB_flowvars_1979_2015.pickle'] #filenames of pickle files which contain AB/BC streamflow data
basinspickle = 'AB_BC_basins2_1979_2015.pickle' #filename of pickle file which contains the basin outlines

In [ ]:
#load data

colabDataPath = '/content/drive/My Drive/Colab Notebooks/T_P_F_pca_lstm/'

pickle_in = open(colabDataPath + basinspickle, 'rb')
stationBasins = pickle.load(pickle_in)

flowDicts = []
for flowfile in flowpickle:
  pickle_in = open(colabDataPath + flowfile,'rb')
  flowDicts.append(pickle.load(pickle_in))

flowDict = {
    'stationID' : np.hstack((flowDicts[0]['stationID'],flowDicts[1]['stationID'])),
    'stationName' : np.hstack((flowDicts[0]['stationName'],flowDicts[1]['stationName'])),
    'stationLat' : np.hstack((flowDicts[0]['stationLat'],flowDicts[1]['stationLat'])),
    'stationLon' : np.hstack((flowDicts[0]['stationLon'],flowDicts[1]['stationLon'])),
    'stationDrainageArea' : np.hstack((flowDicts[0]['stationDrainageArea'],flowDicts[1]['stationDrainageArea'])),
    'all_flowseason' : np.vstack((flowDicts[0]['all_flowseason'],flowDicts[1]['all_flowseason'])),
    'all_flowseason_NF' : np.vstack((flowDicts[0]['all_flowseason_NF'],flowDicts[1]['all_flowseason_NF'])),
    'all_flow' : np.vstack((flowDicts[0]['all_flow'],flowDicts[1]['all_flow'])),
    'all_flow_NF' : np.vstack((flowDicts[0]['all_flow_NF'], flowDicts[1]['all_flow_NF'])),
    'windowDates' : flowDicts[0]['windowDates'],
    'windowYears' : flowDicts[0]['windowYears'],
    'windowMonths' : flowDicts[0]['windowMonths'],
    'windowDays' : flowDicts[0]['windowDays'],
}

pickle_in = open(colabDataPath + 'tempDict.pickle','rb')
tempDict = pickle.load(pickle_in)

pickle_in = open(colabDataPath + 'precDict.pickle','rb')
precDict = pickle.load(pickle_in)

pickle_in = open(colabDataPath + 'relHDict.pickle','rb')
relHDict = pickle.load(pickle_in)

pickle_in = open(colabDataPath + 'ssrdDict.pickle','rb')
ssrdDict = pickle.load(pickle_in)

#unpack data
stationLat = flowDict['stationLat']
stationLon = flowDict['stationLon']
eraLat = tempDict['latERA']
eraLon = tempDict['lonERA']

flowDays = flowDict['windowDays']
flowMonths = flowDict['windowMonths']
flowYears = flowDict['windowYears']
eraDays = tempDict['daysERA']
eraMonths = tempDict['monthsERA']
eraYears = tempDict['yearsERA']

F = flowDict['all_flow_NF'] #discharge with nans filled (NF)
Tmax = tempDict['Tmax']
Tmin = tempDict['Tmin']
P = precDict['P']

In [ ]:
#select subset of stations

maxLat = 56.
stationInds = np.squeeze(np.argwhere(np.expand_dims(stationLat,1)<maxLat)[:,0])

F = np.asarray(F)
F = np.transpose(np.squeeze(F[stationInds]))

stationBasins = [stationBasins[ii] for ii in stationInds]

In [ ]:
#reduce spatial extent to only bound the stations of interest (reduce memory requirements)

bounding_box = 1 #1/0 yes/no if you want to reduce the spatial extent
border = 1 #number of pixels to border the outermost stations

if bounding_box:

  minLon = np.min(stationLon[stationInds])
  maxLon = np.max(stationLon[stationInds])
  minLat = np.min(stationLat[stationInds])
  maxLat = np.max(stationLat[stationInds])

  indMinLonERA = np.argmin(np.abs(eraLon - minLon))
  indMaxLonERA = np.argmin(np.abs(eraLon - maxLon))
  indMinLatERA = np.argmin(np.abs(eraLat - minLat))
  indMaxLatERA = np.argmin(np.abs(eraLat - maxLat))

  if indMinLatERA + border > len(eraLat) - 1:
    indMinLatERA = len(eraLat) - 1
  else:
    indMinLatERA = indMinLatERA + border

  if indMaxLatERA - border < 1:
    indMaxLatERA = 0
  else:
    indMaxLatERA = indMaxLatERA - border + 1

  if indMaxLonERA + border > len(eraLon) - 1:
    indMaxLonERA = len(eraLon) - 1
  else:
    indMaxLonERA = indMaxLonERA + border

  if indMinLonERA - border < 1:
    indMinLonERA = 0
  else:
    indMinLonERA = indMinLonERA - border

  Tmax = Tmax[:, indMaxLatERA:indMinLatERA+1, indMinLonERA:indMaxLonERA+1]
  Tmin = Tmin[:, indMaxLatERA:indMinLatERA+1, indMinLonERA:indMaxLonERA+1]
  P = P[:, indMaxLatERA:indMinLatERA+1, indMinLonERA:indMaxLonERA+1]

  d_eraLon = eraLon[1] - eraLon[0]
  d_eraLat = eraLat[0] - eraLat[1]

  extentERA = [eraLon[indMinLonERA] - d_eraLon/2,eraLon[indMaxLonERA] + d_eraLon/2,eraLat[indMinLatERA] - d_eraLat/2,eraLat[indMaxLatERA] + d_eraLat/2]
  eraLon = eraLon[indMinLonERA:indMaxLonERA+1]
  eraLat = eraLat[indMaxLatERA:indMinLatERA+1]

In [ ]:
#standardize data

#indices of testing/training
trainStartYear = 1979
trainFinYear = 1998
valStartYear = 1999
valFinYear = 2004
testStartYear = 2005
testFinYear = 2010

trainInds = np.squeeze(np.argwhere((flowYears>=trainStartYear) & (flowYears<=trainFinYear)))
valInds = np.squeeze(np.argwhere((flowYears>=valStartYear) & (flowYears<=valFinYear)))
testInds = np.squeeze(np.argwhere((flowYears>=testStartYear) & (flowYears<=testFinYear)))
Ntrain = len(trainInds)
Nval = len(valInds)
Ntest = len(testInds)

#standardize weather variables (normalize wrt training period), then save as np.single for memory
Tmaxmean_train = np.mean([Tmax[trainInds[ii]] for ii in range(len(trainInds))])
Tmaxstd_train = np.std([Tmax[trainInds[ii]] for ii in range(len(trainInds))])
Tmaxnorm = (Tmax - Tmaxmean_train)/Tmaxstd_train
Tmaxnorm = np.single(Tmaxnorm)

Tminmean_train = np.mean([Tmin[trainInds[ii]] for ii in range(len(trainInds))])
Tminstd_train = np.std([Tmin[trainInds[ii]] for ii in range(len(trainInds))])
Tminnorm = (Tmin - Tminmean_train)/Tminstd_train
Tminnorm = np.single(Tminnorm)

Pmean_train = np.mean([P[trainInds[ii]] for ii in range(len(trainInds))])
Pstd_train = np.std([P[trainInds[ii]] for ii in range(len(trainInds))])
Pnorm = (P - Pmean_train)/Pstd_train
Pnorm = np.single(Pnorm)

#normalize flow wrt to training period
Fnorm = np.empty_like(F)
for station in range(np.shape(F)[1]):
    Fnorm[:,station] = (F[:,station] - np.mean(F[:,station]))/np.std(F[:,station])

